Here are the implementations for **Grid Search, Random Search, and Bayesian Search** for hyperparameter tuning using **Scikit-Learn** and **Optuna**.

---

### **1. Grid Search (Exhaustive Search)**
Grid Search systematically tests all possible hyperparameter combinations.

```python
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Load dataset
X, y = load_iris(return_X_y=True)

# Define model
model = RandomForestClassifier()

# Define hyperparameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Perform Grid Search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
```

---

### **2. Random Search (Randomized Parameter Search)**
Random Search randomly samples hyperparameters instead of testing all combinations.

```python
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define hyperparameter distribution
param_dist = {
    'n_estimators': np.arange(10, 200, 10),
    'max_depth': np.arange(3, 20, 1),
    'min_samples_split': np.arange(2, 20, 1)
}

# Perform Random Search
random_search = RandomizedSearchCV(model, param_dist, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)
```

---

### **3. Bayesian Optimization (Using Optuna)**
Bayesian Search uses past evaluations to suggest better hyperparameters iteratively.

```python
import optuna
from sklearn.model_selection import cross_val_score

# Define objective function
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200, step=10)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
    return score

# Perform Bayesian Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Best parameters and score
print("Best Parameters:", study.best_params)
print("Best Score:", study.best_value)
```

---

### **Comparison**
| Method | Exploration | Computation Time | Efficiency |
|--------|------------|-----------------|------------|
| **Grid Search** | Exhaustive | High | Inefficient for large spaces |
| **Random Search** | Random | Medium | Good for large spaces |
| **Bayesian Search** | Adaptive | Low-Medium | Best for optimizing efficiently |

Let me know if you need any modifications or explanations! 🚀

## **Bayesian Optimization for Hyperparameter Tuning**
Bayesian Optimization is an advanced method for **hyperparameter tuning** that intelligently explores the search space by learning from past evaluations. Unlike **Grid Search** (which exhaustively tests all possibilities) and **Random Search** (which picks random combinations), Bayesian Optimization **predicts** promising hyperparameter values based on previous results.

---

## **Why Bayesian Optimization?**
Traditional methods like Grid Search and Random Search become inefficient when dealing with high-dimensional hyperparameter spaces. Bayesian Optimization:
1. **Uses Prior Knowledge** → Learns from past trials to find better hyperparameters.
2. **Explores Intelligently** → Focuses on promising areas of the search space.
3. **Faster Convergence** → Requires fewer evaluations compared to brute-force methods.

---

## **How Bayesian Optimization Works**
Bayesian Optimization follows these key steps:

1. **Define an Objective Function**  
   - This is the function we want to optimize (e.g., accuracy, F1-score).
   - The function takes hyperparameters as input and returns a score.

2. **Build a Probabilistic Model (Surrogate Function)**  
   - Uses a **Gaussian Process (GP)** to estimate the objective function.
   - This is a statistical model that predicts the score for unseen hyperparameters.

3. **Select the Next Hyperparameter Set (Acquisition Function)**  
   - Uses an **acquisition function** to decide the next best set of hyperparameters to evaluate.
   - Balances **exploration** (testing new areas) and **exploitation** (focusing on promising regions).

4. **Update the Model**  
   - After evaluating the new hyperparameters, the GP model is updated.
   - This cycle continues until a stopping criterion is met (e.g., max trials reached).

---

## **Implementation Using Optuna**
Now, let's see **Bayesian Optimization in action** using the **Optuna** library.

### **Step 1: Install Optuna (if not installed)**
```bash
pip install optuna
```

---

### **Step 2: Define the Optimization Process**
We'll optimize a **Random Forest Classifier** using Bayesian Search.

```python
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris

# Load dataset
X, y = load_iris(return_X_y=True)

# Define the objective function
def objective(trial):
    # Define hyperparameter search space
    n_estimators = trial.suggest_int('n_estimators', 10, 200, step=10)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)

    # Initialize the model with suggested hyperparameters
    model = RandomForestClassifier(n_estimators=n_estimators, 
                                   max_depth=max_depth, 
                                   min_samples_split=min_samples_split)

    # Evaluate model performance using cross-validation
    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
    
    return score  # Higher accuracy is better

# Create a study and optimize the hyperparameters
study = optuna.create_study(direction='maximize')  # We want to maximize accuracy
study.optimize(objective, n_trials=20)  # Run for 20 trials

# Print the best hyperparameters and score
print("Best Parameters:", study.best_params)
print("Best Score:", study.best_value)
```

---

## **Explanation of the Code**
- **`trial.suggest_int('n_estimators', 10, 200, step=10)`**  
  - The search space for `n_estimators` is between **10 and 200**, with a step size of **10**.
- **`trial.suggest_int('max_depth', 3, 20)`**  
  - `max_depth` varies from **3 to 20**.
- **`trial.suggest_int('min_samples_split', 2, 20)`**  
  - The number of samples required to split an internal node ranges from **2 to 20**.
- **`cross_val_score()`**  
  - Performs **5-fold cross-validation** to get a robust accuracy estimate.
- **`optuna.create_study(direction='maximize')`**  
  - We optimize for **maximum accuracy**.
- **`study.optimize(objective, n_trials=20)`**  
  - Runs **20 trials**, evaluating different hyperparameter sets.

---

## **Visualization of Optimization**
Optuna provides built-in tools for visualizing the optimization process.

### **Plot the Optimization History**
```python
import optuna.visualization as vis

vis.plot_optimization_history(study)
```

### **Plot Hyperparameter Importances**
```python
vis.plot_param_importances(study)
```

---

## **Comparison with Other Methods**
| Method | **Exploration Strategy** | **Computational Cost** | **Efficiency** |
|--------|------------------------|--------------------|-------------|
| **Grid Search** | Exhaustive | High | Low for large spaces |
| **Random Search** | Random Sampling | Medium | Good for large spaces |
| **Bayesian Optimization** | Smart Exploration | Low-Medium | Best for large spaces |

- **Grid Search**: Tests all possible hyperparameter combinations → **Computationally expensive** 🚀🔴  
- **Random Search**: Randomly picks values → **Faster but still inefficient** ⏳  
- **Bayesian Optimization**: Learns from previous trials → **Finds the best hyperparameters with fewer trials** ✅🔥  

---

## **Advantages of Bayesian Optimization**
✅ **Faster Convergence** → Finds optimal hyperparameters in fewer iterations.  
✅ **More Efficient** → Focuses on promising areas instead of wasting trials.  
✅ **Works Well with Limited Resources** → Reduces unnecessary evaluations.  

---

## **When to Use Bayesian Optimization?**
✔ If your hyperparameter space is **large and continuous**.  
✔ If **evaluating the model is expensive** (e.g., deep learning).  
✔ If **you need an optimal model with fewer trials**.  

---

## **Conclusion**
Bayesian Optimization (via Optuna) **outperforms Grid Search and Random Search** by intelligently selecting hyperparameter values. It is the best choice for complex machine learning models where **computational efficiency matters**.

Would you like to see a **deep learning example** using TensorFlow/Keras? 🚀

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Load dataset
X, y = load_iris(return_X_y=True)

# Define model
model = RandomForestClassifier()

# Define hyperparameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Perform Grid Search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


Best Parameters: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50}
Best Score: 0.9666666666666668


In [2]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define hyperparameter distribution
param_dist = {
    'n_estimators': np.arange(10, 200, 10),
    'max_depth': np.arange(3, 20, 1),
    'min_samples_split': np.arange(2, 20, 1)
}

# Perform Random Search
random_search = RandomizedSearchCV(model, param_dist, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Best Parameters: {'n_estimators': 110, 'min_samples_split': 2, 'max_depth': 14}
Best Score: 0.9666666666666668


In [5]:
import optuna
from sklearn.model_selection import cross_val_score

# Define objective function
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200, step=10)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
    return score

# Perform Bayesian Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Best parameters and score
print("Best Parameters:", study.best_params)
print("Best Score:", study.best_value)


[I 2025-03-30 10:01:22,152] A new study created in memory with name: no-name-a73410c0-7940-465e-8559-901381399405
[I 2025-03-30 10:01:24,088] Trial 0 finished with value: 0.96 and parameters: {'n_estimators': 190, 'max_depth': 15, 'min_samples_split': 5}. Best is trial 0 with value: 0.96.
[I 2025-03-30 10:01:25,561] Trial 1 finished with value: 0.9666666666666668 and parameters: {'n_estimators': 150, 'max_depth': 19, 'min_samples_split': 14}. Best is trial 1 with value: 0.9666666666666668.
[I 2025-03-30 10:01:27,494] Trial 2 finished with value: 0.96 and parameters: {'n_estimators': 180, 'max_depth': 17, 'min_samples_split': 12}. Best is trial 1 with value: 0.9666666666666668.
[I 2025-03-30 10:01:28,791] Trial 3 finished with value: 0.9533333333333334 and parameters: {'n_estimators': 130, 'max_depth': 15, 'min_samples_split': 5}. Best is trial 1 with value: 0.9666666666666668.
[I 2025-03-30 10:01:29,980] Trial 4 finished with value: 0.9666666666666668 and parameters: {'n_estimators': 1

Best Parameters: {'n_estimators': 150, 'max_depth': 19, 'min_samples_split': 14}
Best Score: 0.9666666666666668


In [4]:
pip install optuna

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/383.6 kB ? eta -:--:--
   --- ------------------------------------ 30.7/383.6 kB 1.3 MB/s eta 0:00:01
   ------ -------------------------------- 61.4/383.6 kB 812.7 kB/s eta 0:00:01
   ---------- --------------------------- 102.4/383.6 kB 837.8 kB/s eta 0:00:01
   ---------------- ----------------------- 153.6/383.6 kB 1.0 MB/s eta 0:00:01
   ------------------- -------------------- 184.3/383.6 kB 1.0 MB/s eta 0:00:01
   ----------------------- ---------------- 225.3/383.6 kB 1.1 MB/s eta 0:00:01
   --------------------------- ------------ 266.2/383.6 kB 1.0 MB/s eta 0:00:01
   --------------------------------- ------ 317.4/383.6 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- - 368.6/383.6 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  378.9/383.6 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 383.6/383.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
